# CustomerDemographic

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

In [ ]:
# Read data file
xls = '/content/KPMG_VI_New_raw_data_update_final.xlsx'
data_cd = pd.read_excel(xls, 'CustomerDemographic')
data_cd.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [ ]:
data_cd.columns

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object')

In [ ]:
# Drop first_name, last_name and default
data_cd.drop(['first_name', 'last_name', 'default'], axis=1, inplace=True)
data_cd.columns

Index(['customer_id', 'gender', 'past_3_years_bike_related_purchases', 'DOB',
       'job_title', 'job_industry_category', 'wealth_segment',
       'deceased_indicator', 'owns_car', 'tenure'],
      dtype='object')

In [ ]:
# check datatypes of columns
data_cd.dtypes

customer_id                                     int64
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                        float64
dtype: object

In [ ]:
# Derive a new column 'Age' from DOB and drop 'DOB'
def from_dob_to_age(born):
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

data_cd['Age'] = data_cd['DOB'].apply(lambda x: from_dob_to_age(x))
data_cd.drop('DOB', axis=1, inplace=True)
data_cd.head(4)


,customer_id,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,F,93,Executive Secretary,Health,Mass Customer,N,Yes,11.0,67.0
1,2,Male,81,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,39.0
2,3,Male,61,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,66.0
3,4,Male,33,NaN,IT,Mass Customer,N,No,7.0,59.0


In [ ]:
# Total rows and columns in dataset
data_cd.shape

(4000, 10)

In [ ]:
# Total null values
data_cd.isnull().sum()

customer_id                              0
gender                                   0
past_3_years_bike_related_purchases      0
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                  87
Age                                     87
dtype: int64

In [ ]:
# Since replacing mean or mode of age makes no sense, we delete rows containing missing age
data_cd.dropna(subset=['Age'], inplace=True)
data_cd.isnull().sum()

customer_id                              0
gender                                   0
past_3_years_bike_related_purchases      0
job_title                              497
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
Age                                      0
dtype: int64

In [ ]:
# Drop rows with atleast 2 missing values
d = np.where(data_cd.isnull().sum(axis=1)>1)
data_cd= data_cd.drop(data_cd.index[d])
data_cd.isnull().sum(), data_cd.shape

(customer_id                              0
 gender                                   0
 past_3_years_bike_related_purchases      0
 job_title                              392
 job_industry_category                  551
 wealth_segment                           0
 deceased_indicator                       0
 owns_car                                 0
 tenure                                   0
 Age                                      0
 dtype: int64, (3808, 10))

In [ ]:
data_cd.dropna(subset=['job_industry_category'], inplace=True)
data_cd.isnull().sum(), data_cd.shape

(customer_id                              0
 gender                                   0
 past_3_years_bike_related_purchases      0
 job_title                              392
 job_industry_category                    0
 wealth_segment                           0
 deceased_indicator                       0
 owns_car                                 0
 tenure                                   0
 Age                                      0
 dtype: int64, (3257, 10))

In [ ]:
data_cd.job_industry_category.unique()

array(['Health', 'Financial Services', 'Property', 'IT', 'Retail',
       'Argiculture', 'Manufacturing', 'Telecommunications',
       'Entertainment'], dtype=object)

In [ ]:
# Make a list containing the mode of job_title for each type of job_industry_category
cat_list = data_cd.groupby('job_industry_category').apply(lambda x: x['job_title'].value_counts().idxmax())
cat_list


job_industry_category
Argiculture                            Executive Secretary
Entertainment                      Assistant Media Planner
Financial Services                         Cost Accountant
Health                                       Social Worker
IT                    Business Systems Development Analyst
Manufacturing                            Chemical Engineer
Property                               Account Coordinator
Retail                                Sales Representative
Telecommunications                               Recruiter
dtype: object

In [ ]:
# Iterate through each row of dataframe and replace null job_title with values from cat_list
for ind, row in data_cd.iterrows():
  if pd.isnull(data_cd.at[ind, 'job_title']):
    cat =data_cd.at[ind , 'job_industry_category']
    data_cd.at[ind, 'job_title'] = cat_list[cat]

#for ind in data_cd[:10].index:
#  print(data_cd['job_title'][ind], data_cd['job_industry_category'][ind])
data_cd.isnull().sum()  

customer_id                            0
gender                                 0
past_3_years_bike_related_purchases    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
owns_car                               0
tenure                                 0
Age                                    0
dtype: int64

In [ ]:
data_cd.shape

(3257, 10)

In [ ]:
# find unique values for each column and check for inconsistancies
for col in data_cd:
  print(data_cd[col].unique())

[   1    2    3 ... 3996 3997 3999]
['F' 'Male' 'Female' 'U' 'Femal']
[93 81 61 33 35  6 97 49 99 58 38 85 91 76 72 74 79 55 12 37  5 62 18  3
 17 59 40 46 64 24 63 51 68 57 22  2 48 44 26 47 73 21 67 78 30 28 20 11
 75 41 69 98 16 19 80 83 25 54 23 88 10 77 82 87 27 94 53 32 34  1  9 36
  4 90 95 39  7 42 13 45 71 56 50 14 89 84 96 65 70 31 60 66  0 43 86 92
  8 29 15 52]
['Executive Secretary' 'Administrative Officer' 'Recruiting Manager'
 'Business Systems Development Analyst' 'Sales Representative'
 'Cost Accountant' 'Senior Quality Engineer' 'Account Coordinator'
 'Nuclear Power Engineer' 'Developer I' 'Account Executive'
 'Junior Executive' 'Geological Engineer' 'Project Manager'
 'Safety Technician I' 'Research Assistant I' 'Accounting Assistant III'
 'Editor' 'Research Nurse' 'Safety Technician III' 'Staff Accountant III'
 'Legal Assistant' 'Information Systems Manager' 'Social Worker'
 'Senior Cost Accountant' 'Assistant Media Planner'
 'Payment Adjustment Coordinator' 'Food C

In [ ]:
# Solve inconsistancy in gender
data_cd['gender'] = data_cd['gender'].replace(['F', 'Femal', 'Female'], 'F')
data_cd['gender'] = data_cd['gender'].replace(['M', 'Male'], 'M')
data_cd['gender'].unique()

array(['F', 'M', 'U'], dtype=object)

# CustomerAddress

In [ ]:
data_ca = pd.read_excel(xls, 'CustomerAddress')
data_ca.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [ ]:
data_ca.dtypes

customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object

In [ ]:
# Check for null values in dataset
data_ca.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [ ]:
# Solve inconsistancy in state
data_ca['state'].unique()

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [ ]:
data_ca['state'] = data_ca['state'].replace('New South Wales', 'NSW')
data_ca['state'] = data_ca['state'].replace('Victoria', 'VIC')
data_ca['state'].unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

In [ ]:
data_ca['country'].unique()

array(['Australia'], dtype=object)

# Transactions

In [ ]:
data_t = pd.read_excel(xls, 'Transactions')
data_t.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [ ]:
data_t.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object

In [ ]:
#data_t.drop('product_first_sold_date', axis=1, inplace=True)
#data_t.columns

In [ ]:
data_t.shape

(20000, 13)

In [ ]:
data_t.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [ ]:
# Drop rows with null values in brand, product_line, product_class, product_size and standard cost
d = np.where(data_t.isnull().sum(axis=1)>3)
data_t= data_t.drop(data_t.index[d])
data_t.isnull().sum(), data_t.shape

(transaction_id               0
 product_id                   0
 customer_id                  0
 transaction_date             0
 online_order               358
 order_status                 0
 brand                        0
 product_line                 0
 product_class                0
 product_size                 0
 list_price                   0
 standard_cost                0
 product_first_sold_date      0
 dtype: int64, (19803, 13))

In [ ]:
# Convert Product_first_sold_date from excel time to datetime
import datetime

def convertdate(x):
 serial = x
 seconds = (serial - 25569) * 86400.0
 return datetime.datetime.utcfromtimestamp(seconds)

data_t['product_first_sold_date'] = data_t['product_first_sold_date'].apply(lambda x:convertdate(x))

data_t.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [ ]:
# Drop rows with missing online_order
data_t.dropna(subset=['online_order'], inplace=True)
data_t.isnull().sum(), data_t.shape

(transaction_id             0
 product_id                 0
 customer_id                0
 transaction_date           0
 online_order               0
 order_status               0
 brand                      0
 product_line               0
 product_class              0
 product_size               0
 list_price                 0
 standard_cost              0
 product_first_sold_date    0
 dtype: int64, (19445, 13))

In [ ]:
# Create a new column Profit from list_price and standard cost
data_t['profit'] = data_t['list_price'] - data_t['standard_cost']
data_t.head(3)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,17.87
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,1702.55
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,1544.61


In [ ]:
data_t['online_order'] = data_t['online_order'].astype(int)
data_t.head(3)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit
0,1,2,2950,2017-02-25,0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,17.87
1,2,3,3120,2017-05-21,1,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,1702.55
2,3,37,402,2017-10-16,0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,1544.61


In [ ]:
data_t['order_status'].unique(), data_t['product_line'].unique(), data_t['product_class'].unique(), data_t['product_size'].unique()

(array(['Approved', 'Cancelled'], dtype=object),
 array(['Standard', 'Road', 'Mountain', 'Touring'], dtype=object),
 array(['medium', 'low', 'high'], dtype=object),
 array(['medium', 'large', 'small'], dtype=object))

# NewCustomerList

In [ ]:
data_new = pd.read_excel(xls, 'NewCustomerList')
data_new.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1.05,1.3125,1.640625,1.394531,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,0.59,0.5900,0.737500,0.626875,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,0.89,0.8900,0.890000,0.890000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,0.59,0.7375,0.737500,0.737500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,1.08,1.0800,1.350000,1.350000,4,4,1.703125


In [ ]:
data_new.shape, data_new.dtypes

((1000, 23), first_name                                     object
 last_name                                      object
 gender                                         object
 past_3_years_bike_related_purchases             int64
 DOB                                    datetime64[ns]
 job_title                                      object
 job_industry_category                          object
 wealth_segment                                 object
 deceased_indicator                             object
 owns_car                                       object
 tenure                                          int64
 address                                        object
 postcode                                        int64
 state                                          object
 country                                        object
 property_valuation                              int64
 Unnamed: 16                                   float64
 Unnamed: 17                                   float6

In [ ]:
# Drop Unnamed:16 to 20
data_new.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis=1, inplace=True)
data_new.shape, data_new.columns

((1000, 18), Index(['first_name', 'last_name', 'gender',
        'past_3_years_bike_related_purchases', 'DOB', 'job_title',
        'job_industry_category', 'wealth_segment', 'deceased_indicator',
        'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
        'property_valuation', 'Rank', 'Value'],
       dtype='object'))

In [ ]:
# Derive Age from DOB and drop DOB
def from_dob_to_age(born):
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

data_new['Age'] = data_new['DOB'].apply(lambda x: from_dob_to_age(x))
data_new.drop('DOB', axis=1, inplace=True)
data_new.head(4)

,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,Age
0,Chickie,Brister,Male,86,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750,63.0
1,Morly,Genery,Male,69,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750,50.0
2,Ardelis,Forrester,Female,10,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750,46.0
3,Lucine,Stutt,Female,64,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,41.0


In [ ]:
# Check forr missing values
data_new.isnull().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
Age                                     17
dtype: int64

In [ ]:
# Replace missing last_name with ' '
data_new['last_name'].fillna(' ', inplace=True)
data_new.isnull().sum()

first_name                               0
last_name                                0
gender                                   0
past_3_years_bike_related_purchases      0
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
Age                                     17
dtype: int64

In [ ]:
# Drop rows with missing job_industry_category
data_new.dropna(subset=['job_industry_category'], inplace=True)
data_new.isnull().sum(), data_new.shape

(first_name                              0
 last_name                               0
 gender                                  0
 past_3_years_bike_related_purchases     0
 job_title                              84
 job_industry_category                   0
 wealth_segment                          0
 deceased_indicator                      0
 owns_car                                0
 tenure                                  0
 address                                 0
 postcode                                0
 state                                   0
 country                                 0
 property_valuation                      0
 Rank                                    0
 Value                                   0
 Age                                    17
 dtype: int64, (835, 18))

In [ ]:
data_new['job_industry_category'].unique()

array(['Manufacturing', 'Property', 'Financial Services', 'Entertainment',
       'Retail', 'IT', 'Telecommunications', 'Health', 'Argiculture'],
      dtype=object)

In [ ]:
# Make a list containing the mode of job_title for each type of job_industry_category
cat_list_new = data_new.groupby('job_industry_category').apply(lambda x: x['job_title'].value_counts().idxmax())
cat_list_new

job_industry_category
Argiculture           Analog Circuit Design manager
Entertainment               Assistant Media Planner
Financial Services                  Cost Accountant
Health                                Social Worker
IT                                  Legal Assistant
Manufacturing                     Chemical Engineer
Property              Analog Circuit Design manager
Retail                         Sales Representative
Telecommunications          Human Resources Manager
dtype: object

In [ ]:
# Iterate through each row of dataframe and replace null job_title with values from cat_list
for ind, row in data_new.iterrows():
  if pd.isnull(data_new.at[ind, 'job_title']):
    cat =data_new.at[ind , 'job_industry_category']
    data_new.at[ind, 'job_title'] = cat_list_new[cat]

data_new.isnull().sum()  

first_name                              0
last_name                               0
gender                                  0
past_3_years_bike_related_purchases     0
job_title                               0
job_industry_category                   0
wealth_segment                          0
deceased_indicator                      0
owns_car                                0
tenure                                  0
address                                 0
postcode                                0
state                                   0
country                                 0
property_valuation                      0
Rank                                    0
Value                                   0
Age                                    17
dtype: int64

# Dataframe to excel sheet

In [ ]:
writer = pd.ExcelWriter('kpmg_clean.xlsx')

# Write each dataframe to a different worksheet.
data_cd.to_excel(writer, sheet_name='CustomerDemographic')
data_ca.to_excel(writer, sheet_name='CustomerAddress')
data_t.to_excel(writer, sheet_name='Transactions')

# Close the Pandas Excel writer and output the Excel file.
writer.save()